In [ ]:
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder
from cloth_tools.drake.building import finish_build

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
table_urdf = "table.urdf"

arm_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
gripper_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_wrist_frame = plant.GetFrameByName("wrist_3_link", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
table_frame = plant.GetFrameByName("base_link", table_index)

plant.WeldFrames(world_frame, arm_frame)
plant.WeldFrames(arm_wrist_frame, gripper_frame)
plant.WeldFrames(world_frame, table_frame, RigidTransform([0, 0, 0]))

diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
from typing import List
from airo_typing import JointConfigurationType
import time


def publish_ik_solutions(solutions: List[JointConfigurationType]):
    plant = diagram.plant()
    plant_context = plant.GetMyContextFromRoot(context)

    # meshcat.DeleteRecording() # Doesn't seem necessary, old one is overwritten
    fps = 60.0
    solution_show_time = 2.0
    meshcat.StartRecording(set_visualizations_while_recording=False, frames_per_second=fps)

    t = 0.0

    for joint_configuration in solutions:
        for _ in range(int(solution_show_time * fps)):
            context.SetTime(t)
            plant.SetPositions(plant_context, arm_index, joint_configuration.squeeze())
            diagram.ForcedPublish(context)
            t += 1.0 / fps

    meshcat.StopRecording()
    meshcat.PublishRecording()

In [ ]:
import numpy as np
from airo_typing import Vector3DType
from pydrake.geometry import Cylinder, Meshcat, Rgba
from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.multibody.tree import FrameIndex
from pydrake.trajectories import BsplineTrajectory


def add_meshcat_triad(meshcat, path, length=0.05, radius=0.002, opacity=1.0, X_W_Triad=RigidTransform(), rgba_xyz=None):
    if rgba_xyz is None:
        rgba_xyz = [[1, 0, 0, opacity], [0, 1, 0, opacity], [0, 0, 1, opacity]]

    meshcat.SetTransform(path, X_W_Triad)
    # x-axis
    X_TG = RigidTransform(RotationMatrix.MakeYRotation(np.pi / 2), [length / 2.0, 0, 0])
    meshcat.SetTransform(path + "/x-axis", X_TG)
    meshcat.SetObject(path + "/x-axis", Cylinder(radius, length), Rgba(*rgba_xyz[0]))

    # y-axis
    X_TG = RigidTransform(RotationMatrix.MakeXRotation(np.pi / 2), [0, length / 2.0, 0])
    meshcat.SetTransform(path + "/y-axis", X_TG)
    meshcat.SetObject(path + "/y-axis", Cylinder(radius, length), Rgba(*rgba_xyz[1]))

    # z-axis
    X_TG = RigidTransform([0, 0, length / 2.0])
    meshcat.SetTransform(path + "/z-axis", X_TG)
    meshcat.SetObject(path + "/z-axis", Cylinder(radius, length), Rgba(*rgba_xyz[2]))


In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
from airo_spatial_algebra import SE3Container

transform = RigidTransform(p=[0.25, 0, 0.4], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

with np.printoptions(precision=10, suppress=True):
    print("tcp_pose_0")
    print(tcp_pose_0)


In [ ]:
from ur_analytic_ik import ur5e

solutions = ur5e.inverse_kinematics_with_tcp(tcp_pose_0, tcp_transform)
publish_ik_solutions(solutions)

In [ ]:
tcp_pose_1 = np.identity(4)
X = np.array([-1.0, 0.0, 0.0])
Y = np.array([0.0, 1.0, 0.0])
Z = np.array([0.0, 0.0, -1.0])
top_down_orientation = np.column_stack([X, Y, Z])
translation = np.array([-0.2, -0.2, 0.2])

tcp_pose_1[:3, :3] = top_down_orientation
tcp_pose_1[:3, 3] = translation

with np.printoptions(precision=3, suppress=True):
    print("tcp_pose_1")
    print(tcp_pose_1)

In [ ]:
add_meshcat_triad(meshcat, "tcp_pose_1", X_W_Triad=RigidTransform(tcp_pose_1))

In [ ]:
with np.printoptions(precision=3, suppress=True):
    print("inverse_kinematics")
    print(ur5e.inverse_kinematics(tcp_pose_1))


In [ ]:
publish_ik_solutions(ur5e.inverse_kinematics_with_tcp(tcp_pose_1, tcp_transform))